<a href="https://colab.research.google.com/github/rswijesena/AI/blob/main/Basic_Image_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the libraries
import torch, pdb
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import matplotlib.pyplot as plt


In [2]:
# visualization function
def show(tensor, ch=1, size=(28,28), num=16):
  # tensor 128 x 784
  data = tensor.detach().cpu().view(-1, ch, *size)
  grid = make_grid(data[:num], nrows = 4).permute(1,2,0)
  plt.imshow(grid)
  plt.show()


In [ ]:
# setup if the main parameters and hyperparameters
epochs = 300